In [3]:
import sys
import itertools
import os

import numpy as np
import pandas as pd
import scipy
from scipy.signal import savgol_filter

import neuroseries as nts

from progressbar import ProgressBar

import time
import matplotlib.pyplot as plt


import seaborn as sns 


import bk.load
import bk.signal
import bk.plot


import importlib; 
importlib.reload(bk.signal)
importlib.reload(bk.plot)
%matplotlib qt
import basefunction.anass_load as at

In [5]:
def computeRateMap(neuron,pos,nxbins=10,nybins=10,plot= False):
    
    """
        neuron : as Tsd from neuroseries
        pos : as Tsd from neuroseries
        
        output : 
        rate_map : Ratemap in Hz 
        spike_count : number of spikes per spatial bin
        occupancy : number of pos in spatial bin
    """
    
    
    dt = scipy.stats.mode(np.diff(pos.as_units('s').index))[0][0]
    
    pos_spike = pos.realign(neuron)
    
    xbins = np.linspace(pos.x.min(),pos.x.max()+0.01,nxbins)
    ybins = np.linspace(pos.y.min(),pos.y.max()+0.01,nybins)
    
    spike_count,_,_ = np.histogram2d(pos_spike.y,pos_spike.x,[ybins,xbins])
    occupancy,_,_ = np.histogram2d(pos.y,pos.x,[ybins,xbins])
    
    rate_map = (spike_count/(occupancy+1))/dt
    rate_map = pd.DataFrame(index = ybins[0:-1]+np.diff(ybins)/2,columns=xbins[0:-1]+np.diff(xbins)/2, data = rate_map)
    
    
    if plot:
        plt.figure()
        plt.subplot(211)

        plt.plot(pos['x'], pos['y'], alpha = 0.5)
        plt.scatter(pos_spike['x'], pos_spike['y'], color = 'red', alpha = 0.5, edgecolor = None)
        for i in xbins: plt.axvline(i)
        for i in ybins: plt.axhline(i)
        plt.xlim(xbins[0], xbins[-1])
        plt.ylim(ybins[0], ybins[-1])
        plt.subplot(212)
        plt.imshow(rate_map, origin = 'lower', aspect= 'auto')
        plt.clim([0,10])
    
    
    return rate_map,spike_count,occupancy


def spatialInformationScore(spike_count,occupancy):
    
    """
        Initial code by KevinAllen
        Adapted by BK 2/11/20
        
        rate_map : number of spike per spatial bin
        occupancy : number of position per spatial bin
    """
    
    
    spike_count = spike_count[occupancy>0]
    occupancy = occupancy[occupancy>0]
    
    rate_map = spike_count/occupancy
    
    mRate = spike_count.sum()/occupancy.sum()
    prob = occupancy/occupancy.sum()
    
    index = rate_map>0
    
    info = np.sum(prob[index] * (rate_map[index]/mRate) * np.log2(rate_map[index]/mRate))
    return info


In [7]:
bk.load.path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
bk.load.session = 'Rat08-20130713'
os.chdir(bk.load.path)

session = 'Rat08-20130713'
neurons,metadata = bk.load.loadSpikeData(bk.load.path)

run = scipy.io.loadmat(bk.load.path + 'runintervals.mat')
run = nts.IntervalSet(run['runintervals'][:,0],run['runintervals'][:,1],time_units='s')

Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [58]:
# path = bk.load.sessions()['Path'][4]
# bk.load.current_session(path)

os.chdir(bk.load.path)

neurons,metadata = bk.load.loadSpikeData(bk.load.path)
pos = bk.load.pos()
states = bk.load.states()

run = scipy.io.loadmat('runintervals.mat')
run = nts.IntervalSet(run['runintervals'][:,0],run['runintervals'][:,1],time_units='s')

trackrun = scipy.io.loadmat(bk.load.session+'-TrackRunTimes.mat')
trackrun = nts.IntervalSet(trackrun['trackruntimes'][:,0],trackrun['trackruntimes'][:,1],time_units='s')

pos_run = pos.restrict(run).restrict(trackrun)

Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [8]:
theta = scipy.io.loadmat('Billel/ThetaMod/ThetaMod.mat')

In [9]:
for s,i in zip(metadata['Shank'],metadata['Id']):
    index = np.all([s,i] == theta['metadata'][:,2:4],1)
    if not np.any(index):
        print(s,i)

IndexError: index 1 is out of bounds for axis 1 with size 1

In [59]:
plt.bar(theta['binned']['hpc'][0][0].squeeze(),theta['dist']['hpc'][0][0][:,21].squeeze())

<BarContainer object of 100 artists>

In [72]:
info = np.array(info).squeeze()
MRL = np.array(MRL)
data = {'MRL':np.log(MRL),'info':info,'region':metadata['Region'],'type':metadata['Type']}
data = pd.DataFrame(data)
sns.scatterplot('info','MRL',data = data,hue = 'region',style = 'type')

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


<AxesSubplot:xlabel='info', ylabel='MRL'>

In [123]:
info = []
rate_maps = []
for n,s,i in zip(neurons,metadata['Shank'],metadata['Id']):
    m,spike_count,occupancy = computeRateMap(n,pos_run,100,10,False)
    rate_maps.append(m)
    info.append(spatialInformationScore(spike_count,occupancy))
MRL = theta['stats']['hpc'][0][0]['r'][0][0].squeeze()

metadata['info'] = info
plt.cla()
sns.violinplot(y = 'info',data = metadata,x = 'Region')

<AxesSubplot:xlabel='Region', ylabel='info'>

In [104]:
plt.cla()
plt.imshow(rate_maps[204],origin='lower',aspect='auto')
plt.clim([0,0.01])

In [101]:
info[44] = np.nan

In [90]:
info = np.array(info)

In [102]:
plt.bar()

204

In [118]:
computeRateMap(neurons[5],pos_run,120,20,True)
plt.colorbar()

0.025600000000849832


In [2]:
theta['dist']

NameError: name 'theta' is not defined

In [28]:
MRL = theta['stats']['hpc'][0][0]['r'][0][0].squeeze()

In [30]:
np.argmax(MRL)

21

In [38]:
theta['binned'][]

array([(array([[0.03141593],
       [0.09424778],
       [0.15707963],
       [0.21991149],
       [0.28274334],
       [0.34557519],
       [0.40840704],
       [0.4712389 ],
       [0.53407075],
       [0.5969026 ],
       [0.65973446],
       [0.72256631],
       [0.78539816],
       [0.84823002],
       [0.91106187],
       [0.97389372],
       [1.03672558],
       [1.09955743],
       [1.16238928],
       [1.22522113],
       [1.28805299],
       [1.35088484],
       [1.41371669],
       [1.47654855],
       [1.5393804 ],
       [1.60221225],
       [1.66504411],
       [1.72787596],
       [1.79070781],
       [1.85353967],
       [1.91637152],
       [1.97920337],
       [2.04203522],
       [2.10486708],
       [2.16769893],
       [2.23053078],
       [2.29336264],
       [2.35619449],
       [2.41902634],
       [2.4818582 ],
       [2.54469005],
       [2.6075219 ],
       [2.67035376],
       [2.73318561],
       [2.79601746],
       [2.85884931],
       [2.92168117],
     